<a href="https://colab.research.google.com/github/marco-siino/SemEval2024/blob/main/Task%2010/SemEval_2024_Task_10_Mistral7B_MSiino.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Installing dependencies. You might need to tweak the CMAKE_ARGS for the `llama-cpp-python` pip package.

In [1]:
# GPU llama-cpp-python; Starting from version llama-cpp-python==0.1.79, it supports GGUF
!CMAKE_ARGS="-DLLAMA_CUBLAS=on " pip install 'llama-cpp-python>=0.1.79' --force-reinstall --upgrade --no-cache-dir
# For download the models
!pip install huggingface_hub
!pip install datasets
!pip install -U deep-translator

import datasets
from datasets import load_dataset
from deep_translator import GoogleTranslator
import json
import re
import random
import numpy as np
import tqdm.notebook as tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.7/10.7 MB 147.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 283.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.5/45.5 kB 186.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.2/133.2 kB 276.8 MB/s eta 0:00:00
  Created wheel for llama-cpp-python: filename=llama_cpp_python-0.2.38-cp310-cp310-manylinux_2_35_x86_64.whl size=9660884 sha256=df1d82865729b2b5e98a7d394071d77e46dc9c6da110dc03c855f2e76251afab
  Stored in directory: /tmp/pip-ephem-wheel-cache-cnoau6j6/wheels/eb/58/77/20d3d9a235b4930050fbcde1ad4f0a4d054644269e801b08aa
Successfully built llama-cpp-python
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.5.0
    Uninstalling typing_extensions-4.

Downloading an instruction-finetuned Mistral model.

In [2]:
from huggingface_hub import hf_hub_download

model_name_or_path = "TheBloke/Mistral-7B-Instruct-v0.2-GGUF"
model_basename = "mistral-7b-instruct-v0.2.Q6_K.gguf"

model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

# This config has been tested on an RTX 3080 (VRAM of 16GB).
# you might need to tweak with respect to your hardware.
from llama_cpp import Llama
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=4, #16, # CPU cores
    n_batch=800, #8000, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32, # Change this value based on your model and your GPU VRAM pool.
    n_ctx=8192, # Context window
    logits_all=True
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


mistral-7b-instruct-v0.2.Q6_K.gguf:   0%|          | 0.00/5.94G [00:00<?, ?B/s]

AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 
Model metadata: {'tokenizer.chat_template': "{{ bos_token }}{% for message in messages %}{% if (message['role'] == 'user') != (loop.index0 % 2 == 0) %}{{ raise_exception('Conversation roles must alternate user/assistant/user/assistant/...') }}{% endif %}{% if message['role'] == 'user' %}{{ '[INST] ' + message['content'] + ' [/INST]' }}{% elif message['role'] == 'assistant' %}{{ message['content'] + eos_token}}{% else %}{{ raise_exception('Only user and assistant roles are supported!') }}{% endif %}{% endfor %}", 'tokenizer.ggml.add_eos_token': 'false', 'tokenizer.ggml.padding_token_id': '0', 'tokenizer.ggml.unknown_token_id': '0', 'tokenizer.ggml.eos_token_id': '2', 'general.architecture': 'llama', 'llama.rope.freq_base': '1000000.000000', 'llama.context_length': '32768', 'general.na

# Download the dataset for the three subtasks.

In [3]:
# Subtask 1.

!wget 'https://raw.githubusercontent.com/marco-siino/SemEval2024/main/Task%2010/dataset/subtask1/MaSaC_train_erc.json'
!wget 'https://raw.githubusercontent.com/marco-siino/SemEval2024/main/Task%2010/dataset/subtask1/MaSaC_val_erc.json'
!wget 'https://raw.githubusercontent.com/marco-siino/SemEval2024/main/Task%2010/dataset/subtask1/MaSaC_test_erc.json'

train_path_1 = 'MaSaC_train_erc.json'
val_path_1 = 'MaSaC_val_erc.json'
test_path_1 = 'MaSaC_test_erc.json'

--2024-02-03 16:48:35--  https://raw.githubusercontent.com/marco-siino/SemEval2024/main/Task%2010/dataset/subtask1/MaSaC_train_erc.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 720262 (703K) [text/plain]
Saving to: ‘MaSaC_train_erc.json’

MaSaC_train_erc.jso 100%[===================>] 703.38K  --.-KB/s    in 0.005s  

2024-02-03 16:48:35 (138 MB/s) - ‘MaSaC_train_erc.json’ saved [720262/720262]

--2024-02-03 16:48:35--  https://raw.githubusercontent.com/marco-siino/SemEval2024/main/Task%2010/dataset/subtask1/MaSaC_val_erc.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTT

In [4]:
# Subtask 2.

!wget 'https://raw.githubusercontent.com/marco-siino/SemEval2024/main/Task%2010/dataset/subtask2/MaSaC_train_efr.json'
!wget 'https://raw.githubusercontent.com/marco-siino/SemEval2024/main/Task%2010/dataset/subtask2/MaSaC_val_efr.json'
!wget 'https://raw.githubusercontent.com/marco-siino/SemEval2024/main/Task%2010/dataset/subtask2/MaSaC_test_efr.json'

train_path_2 = 'MaSaC_train_efr.json'
val_path_2 = 'MaSaC_val_efr.json'
test_path_2 = 'MaSaC_test_efr.json'

--2024-02-03 16:48:36--  https://raw.githubusercontent.com/marco-siino/SemEval2024/main/Task%2010/dataset/subtask2/MaSaC_train_efr.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8644822 (8.2M) [text/plain]
Saving to: ‘MaSaC_train_efr.json’

MaSaC_train_efr.jso 100%[===================>]   8.24M  --.-KB/s    in 0.04s   

2024-02-03 16:48:37 (197 MB/s) - ‘MaSaC_train_efr.json’ saved [8644822/8644822]

--2024-02-03 16:48:37--  https://raw.githubusercontent.com/marco-siino/SemEval2024/main/Task%2010/dataset/subtask2/MaSaC_val_efr.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.


In [5]:
# Subtask 3.

!wget 'https://raw.githubusercontent.com/marco-siino/SemEval2024/main/Task%2010/dataset/subtask3/MELD_train_efr.json'
!wget 'https://raw.githubusercontent.com/marco-siino/SemEval2024/main/Task%2010/dataset/subtask3/MELD_val_efr.json'
!wget 'https://raw.githubusercontent.com/marco-siino/SemEval2024/main/Task%2010/dataset/subtask3/MELD_test_efr.json'

train_path_3 = 'MELD_train_efr.json'
val_path_3 = 'MELD_val_efr.json'
test_path_3 = 'MELD_test_efr.json'

--2024-02-03 16:48:38--  https://raw.githubusercontent.com/marco-siino/SemEval2024/main/Task%2010/dataset/subtask3/MELD_train_efr.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5180775 (4.9M) [text/plain]
Saving to: ‘MELD_train_efr.json’

MELD_train_efr.json 100%[===================>]   4.94M  --.-KB/s    in 0.01s   

2024-02-03 16:48:38 (351 MB/s) - ‘MELD_train_efr.json’ saved [5180775/5180775]

--2024-02-03 16:48:38--  https://raw.githubusercontent.com/marco-siino/SemEval2024/main/Task%2010/dataset/subtask3/MELD_val_efr.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP

# Create few-shot samples from training and validation set.

In [6]:
def create_few_shot_samples_task1(json_set_path,nr_samples):

  nr_few_shot_samples = nr_samples

  few_shot_counter = 0

  few_shot_samples = ''

  with open(json_set_path, 'r') as istr:
    train_1_json = json.load(istr)
  for i in tqdm.trange(len(train_1_json)):
    if few_shot_counter == nr_few_shot_samples:
      break

    for sentence_nr in range(0,len(train_1_json[i]['speakers'])):
      few_shot_counter+=1
      #print(train_1_json[i]['speakers'][sentence_nr])
      translated_utterance = GoogleTranslator(source='hi', target='en').translate(train_1_json[i]['utterances'][sentence_nr])
      few_shot_samples += train_1_json[i]['speakers'][sentence_nr]+' // ' + translated_utterance + ' // ' + train_1_json[i]['emotions'][sentence_nr] + " \n "

      if few_shot_counter == nr_few_shot_samples:
        break

  return few_shot_samples

In [14]:
few_shot_samples = create_few_shot_samples_task1(train_path_1,80)
few_shot_samples += create_few_shot_samples_task1(val_path_1,80)

  0%|          | 0/343 [00:00<?, ?it/s]

  0%|          | 0/46 [00:00<?, ?it/s]

In [15]:
print(few_shot_samples)

maya // What does Indravadan keep in the house? Indravadan please why didn't you throw away all the useless things? // disgust 
 indu // ok, let's rise, let's spring // contempt 
 rosesh // Momma! Hath Chodiye Dad! // anger 
 indu // Look Maya or I am not speaking! Then you will say that you are not throwing the wrong things, come on. // neutral 
 maya // Indravadan, for how many years have you been telling this joke? What is happening today? // neutral 
 indu // joke on kaikai // contempt 
 rosesh // very funny // contempt 
 indu // Maya, Monisha and Sahil are here, see what they are using and see how much they have done. // neutral 
 sahil // hi guys // joy 
 rosesh // hi // joy 
 maya // hi darling. achcha indravadan, please throw away this garbage! // neutral 
 indu // Ok Maya, let's throw the spring at you // neutral 
 sahil // Mom! Diwali lunch at our house // joy 
 monish // And I have made fresh food. // joy 
 maya // i rest my case indravadan. hase donon, huh? achcha ab to ple

# Collect all the emotions in the train and in the dev set for subtask 1.

In [9]:
emotions_list = []

# Train set first.
with open(train_path_1, 'r') as istr:
  file_json = json.load(istr)
  for i in tqdm.trange(len(file_json)):
    for sentence_nr in range(0,len(file_json[i]['emotions'])):
      #print(file_json[i]['emotions'][sentence_nr])
      if file_json[i]['emotions'][sentence_nr] not in emotions_list:
        emotions_list.append(file_json[i]['emotions'][sentence_nr])
      break

# Validation set after.
with open(val_path_1, 'r') as istr:
  file_json = json.load(istr)
  for i in tqdm.trange(len(file_json)):
    for sentence_nr in range(0,len(file_json[i]['emotions'])):
      #print(file_json[i]['emotions'][sentence_nr])
      if file_json[i]['emotions'][sentence_nr] not in emotions_list:
        emotions_list.append(file_json[i]['emotions'][sentence_nr])
      break



  0%|          | 0/343 [00:00<?, ?it/s]

  0%|          | 0/46 [00:00<?, ?it/s]

In [10]:
emotions_list

['disgust',
 'joy',
 'neutral',
 'anger',
 'sadness',
 'contempt',
 'surprise',
 'fear']

# Run!

In [16]:
with open('answer.txt', 'w') as f:
        f.write('')

# simple JSON loading
with open(test_path_1, 'r') as istr:
    data_val_all = json.load(istr)
num_sample = len(data_val_all)
#print(num_sample)


with open(test_path_1, 'r') as istr:
    test_1_json = json.load(istr)
for i in range(0,len(test_1_json)):
  current_sample =''
  for sentence_nr in range(0,len(test_1_json[i]['speakers'])):
      print("\n\n")
      print("ORIGINAL: "+ test_1_json[i]['speakers'][sentence_nr]+' // '+test_1_json[i]['utterances'][sentence_nr])

      try:
        translated_utterance = GoogleTranslator(source='hi', target='en').translate(test_1_json[i]['utterances'][sentence_nr])
      except:
        translated_utterance = test_1_json[i]['utterances'][sentence_nr]

      current_sample = test_1_json[i]['speakers'][sentence_nr]+' // ' + translated_utterance + ' // '

      prompt = "\n Use the CONTEXT to complete the SENTENCE using ONLY one emotion among: disgust, joy, neutral, anger, sadness, contempt, surprise, fear. Do not explain!\n"
      prompt += f"<s>[INST] CONTEXT: {few_shot_samples}\n SENTENCE:{current_sample} [/INST]"

      #print(prompt)

      response = lcpp_llm(
        prompt=prompt,
        temperature= 0.2,
        logprobs=1,
      )

      answer = str(response["choices"][0]["text"]).strip().lower()
      answer = answer.split()[0]
      # Sometime output contains a '.' remove it!
      answer = answer.replace('.','')

      # If the predicted word is not in emotion list just replace with neutral.
      if answer not in emotions_list:
        answer = 'neutral'

      #current_sample += answer + " \n "

      print("GENERATED: "+ current_sample+answer)

      with open('answer.txt', 'a') as f:
        f.write(answer+"\n")





ORIGINAL: maya // kuchh karo sahil please kuchh karo. mera rosesh adopt ho karke chala gaya na to me, i know this sounds horribly melodramatic, monishaish, par me mar jaaungi. i swear mein mar jaaungi


Llama.generate: prefix-match hit


GENERATED: maya // Please go to the beach. My rosesh started adopting me, I know this sounds horribly melodramatic, monishaish, but I will die. i swear i will live in the sea // sadness



ORIGINAL: indu // monisha tum muskura kya rahi ho?


Llama.generate: prefix-match hit


GENERATED: indu // monisha why are you smiling? // neutral



ORIGINAL: monisha // haila! main? main kahan? kya daddy ji?


...

Llama.generate: prefix-match hit


GENERATED: rosesh // When will you come, my life partner? I don't have to wait 51 years. okay // joy



ORIGINAL: rosesh // aye ayi ayi gadi aayi. correct 12 bajne me 2 minute. oh god jaldi se dikha do mere janam janam ke sathi ko


Llama.generate: prefix-match hit


GENERATED: rosesh // aayi aayi aayi gadi aayi. correct 12 minutes in 2 minutes. Oh God, please show me quickly to my beloved and my beloved's companion. // neutral


SyntaxError: 'break' outside loop (<ipython-input-16-7f6e1d2fdbe3>, line 53)

In [19]:
counter=0
for i in range(0,len(test_1_json)):
  for sentence_nr in range(0,len(test_1_json[i]['utterances'])):
    counter+=1

print(counter)

1580


In [31]:
open('newfile','w').writelines([ line for line in open('SemEval_2024_Task_10_Mistral7B_MSiino (1).ipynb') if 'GENERATED:' in line])

result_not_filtered = [ line for line in open('SemEval_2024_Task_10_Mistral7B_MSiino (1).ipynb') if 'GENERATED:' in line]


In [56]:
with open('answer.txt', 'w') as f:
        f.write('')

In [57]:
with open('newfile') as f:
   for line in f:
       # For Python3, use print(line)
       x = line.split()
       result = x[len(x)-1].replace('\\n",','')
       result = result.replace('\\n"','')
       print (result)
       with open('answer.txt', 'a') as f:
        f.write(result+"\n")

sadness
neutral
fear
neutral
anger
disgust
neutral
neutral
neutral
neutral
neutral
neutral
disgust
fear
fear
neutral
neutral
joy
neutral
neutral
contempt
joy
neutral
neutral
sadness
disgust
fear
surprise
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
disgust
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
joy
joy
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
joy
neutral
neutral
neutral
neutral
neutral
fear
joy
neutral
neutral
neutral
joy
neutral
neutral
neutral
fear
disgust
neutral
neutral
neutral
neutral
sadness
sadness
neutral
neutral
neutral
neutral
disgust
neutral
fear
neutral
neutral
fear
neutral
neutral
neutral
neutral
anger
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
fear
neutral
fear
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
neutral
surprise
neutral
joy
neutral
neutral
fear
neutral
neutral
fear
neutral
neutral
neutral
neutral
neutral
fear


In [58]:
# Now fill the remaining lines for the other two tasks.

for i in range(1581,17913):
  r_int_value = random.randint(0, 1)
  r_string_value = str(r_int_value)+'.0'
  with open('answer.txt', 'a') as f:
        f.write(r_string_value+"\n")


In [59]:
!zip -r results.zip answer.txt

  adding: answer.txt (deflated 94%)


In [60]:
from google.colab import files
files.download('results.zip')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>